In [1]:
#!/ usr/bin/env python3 .7
# Copyright 2020 , Gurobi Optimization , LLC
# This example formulates and solves the following simple QP model :
#
# minimize    x + y + x^2 + x*y + y^2 + y*z + z^2
# subject to  x + 2 y + 3 z >= 4
#             x +   y       >= 1
#             x, y, z non-negative
#
# The example illustrates the use of dense matrices to store A and Q
# (and dense vectors for the other relevant data ). We don ’t recommend
# that you use dense matrices , but this example may be helpful if you
# already have your data in this format .

In [2]:
import sys
import gurobipy as gp
from gurobipy import GRB

In [10]:
def dense_optimize(rows, cols, c, Q, A, sense, rhs, lb, ub, vtype, solution):
    
    model = gp.Model()
    
    # Variables
    variables = []
    for j in range(cols):
        variables.append(model.addVar(lb=lb[j], ub=ub[j], vtype=vtype[j]))
        
    # Populate A matrix
    for i in range(rows):
        expr = gp.LinExpr()
        for j in range(cols):
            if A[i][j] != 0:
                expr += A[i][j] * variables[j]
        model.addConstr(expr, sense[i], rhs[i])
    
    # Populate objective
    obj = gp.QuadExpr()
    # Quadratic terms
    for i in range(cols):
        for j in range(cols):
            if Q[i][j] != 0:
                obj += Q[i][j] * variables[i] * variables[j]
    # Linear terms
    for j in range(cols):
        if c[j] != 0:
            obj += c[j] * variables[j]
    model.setObjective(obj)
    
    # Solve
    model.optimize()
    
    # Write to file
    model.write('dense.lp')
    
    if model.status == GRB.OPTIMAL:
        x = model.getAttr('x', variables)
        for i in range(cols):
            solution[i] = x[i]
        return True
    else:
        return False

In [8]:
# Put model data into dense matrices

c = [1, 1, 0]
Q = [[1, 1, 0], [0, 1, 1], [0, 0, 1]]
A = [[1, 2, 3], [1, 1, 0]]
sense = [GRB.GREATER_EQUAL, GRB.GREATER_EQUAL]
rhs = [4, 1]
lb = [0, 0, 0]
ub = [GRB.INFINITY, GRB.INFINITY, GRB.INFINITY]
vtype = [GRB.CONTINUOUS, GRB.CONTINUOUS, GRB.CONTINUOUS]
sol = [0] * 3

In [11]:
# Optimize

success = dense_optimize(2, 3, c, Q, A, sense, rhs, lb, ub, vtype, sol)

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0xb6336beb
Model has 5 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve time: 0.10s
Presolved: 2 rows, 3 columns, 5 nonzeros
Presolved model has 5 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 2
 AA' NZ     : 6.000e+00
 Factor NZ  : 1.000e+01
 Factor Ops : 3.000e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.68862705e+05 -1.66862803e+05  1.50e+03 4.63e-07  9.99e+05     0s
   1   3.32226526e+05 -3.31124005e+05  1.50e-03 4.55e-13  1.33e+05     0s
   2   4.88052576e+04 -4.83822339e+04  1.50e-09 5.68e-14  1.94

In [12]:
if success: 
    print('x: %g, y: %g, z: %g' % (sol[0], sol[1], sol[2]))

x: 0.571429, y: 0.428571, z: 0.857143
